# Détection d'anomalies en temps réel sur les logs

Ce notebook permet d'appliquer un modèle pré-entraîné de détection d'anomalies sur de nouveaux logs système. Il utilise :
- Un modèle Isolation Forest pré-entraîné
- Des encodeurs de caractéristiques sauvegardés
- Un pipeline de prétraitement des données

## 1. Import des bibliothèques nécessaires

In [18]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import OneHotEncoder
import joblib

### 2. Chargement des données d'entrainement

In [4]:
df = pd.read_csv('all_logs.csv')

In [9]:
df.dropna(inplace=True)

In [12]:
df.describe()

,IdProcess
count,91899.000000
mean,39088.466839
std,82738.624822
min,1.000000
25%,1410.000000
50%,2017.000000
75%,60954.000000
max,472349.000000


from matplotlib import pyplot as plt
_df_0['IdProcess'].plot(kind='hist', bins=20, title='IdProcess')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_1['IdProcess'].plot(kind='line', figsize=(8, 4), title='IdProcess')
plt.gca().spines[['top', 'right']].set_visible(False)

### 3. Encodage des colonnes catégoriques

In [10]:
encoder = OneHotEncoder(sparse_output=False)
categorical_features = encoder.fit_transform(df[['Hostname', 'Process']])

### 4. Encodage des messages


In [11]:
vectorizer = CountVectorizer(max_features=500)  # On limite les dimensions
message_features = vectorizer.fit_transform(df['Message']).toarray()

### 5. Combinaison des caractéristiques


In [15]:
features = np.hstack([categorical_features, message_features])

### 6. Modèle d'anomalie simple


In [16]:
model = IsolationForest(contamination=0.01)  # Fraction estimée d'anomalies
model.fit(features)


IsolationForest(contamination=0.01)

### 7. Sauvegarde du modèle

In [19]:
joblib.dump(model, f"model.pkl")
joblib.dump(encoder, f"encoder.pkl")
joblib.dump(vectorizer, f"vectorizer.pkl")

['vectorizer.pkl']

### 7. Prédiction des anomalies


In [27]:
# On charge le modèle précedement entrainé et on effectue les prédictions
def predict_anomalies(data_path, output_path):
    # Charger les données
    df = pd.read_csv(data_path)
    df.dropna(inplace=True)

    # Recharger le modèle et les encoders
    model = joblib.load(f"model.pkl")
    encoder = joblib.load(f"encoder.pkl")
    vectorizer = joblib.load(f"vectorizer.pkl")

    # Prétraitement des nouvelles données
    categorical_features = encoder.transform(df[['Hostname', 'Process']])
    message_features = vectorizer.transform(df['Message']).toarray()
    features = np.hstack([categorical_features, message_features])

    # Prédire les anomalies
    df['Anomaly'] = model.predict(features)  # -1 = anomalie, 1 = normal
    anomaly_count = (df['Anomaly'] == -1).sum()
    print(f"Nombre d'anomalies détectées : {anomaly_count}")

    # On isole les anomalies pour pouvoir les traiter par la suite
    anomalies = df[df['Anomaly'] == -1]
    anomalies.to_csv('only_anomalies.csv', index=False)
    print(f"Anomalies sauvegardées dans : only_anomalies.csv")


    # Sauvegarder les résultats
    df.to_csv(output_path, index=False)
    print(f"Résultats sauvegardés dans : {output_path}")

In [28]:
predict_anomalies('tainted_logs.csv', 'new_logs_with_anomalies.csv')

Nombre d'anomalies détectées : 705
Anomalies sauvegardées dans : only_anomalies.csv
Résultats sauvegardés dans : new_logs_with_anomalies.csv


In [29]:
anomalies_found_df = pd.read_csv("only_anomalies.csv")

In [30]:
anomalies_found_df.head()

,Date,Hostname,Process,IdProcess,Message,Anomaly
0,2024-12-16T14:09:28.373121+01:00,hilbert02,gnome-shell,1456.0,Extension launch-new-instance@gnome-shell-exte...,-1
1,2024-12-16T17:09:41.242558+01:00,hilbert02,gnome-shell,2050.0,"Object Gdm.UserVerifierProxy (0x55bff5588600),...",-1
2,2024-12-18T13:48:08.345449+01:00,hilbert02,gnome-shell,1451.0,Extension launch-new-instance@gnome-shell-exte...,-1
3,2024-12-16T14:09:25.817897+01:00,hilbert02,dbus-daemon,1410.0,[session uid=112 pid=1410] Activating via syst...,-1
4,2024-12-16T14:09:28.373121+01:00,hilbert02,gnome-shell,1456.0,Extension launch-new-instance@gnome-shell-exte...,-1
